In [2]:
import subprocess
import sys

def get_outdated_packages():
    result = subprocess.run([sys.executable, '-m', 'pip', 'list', '--outdated', '--format=freeze'],
                            capture_output=True, text=True)
    packages = []
    for line in result.stdout.splitlines():
        package = line.split('==')[0]
        packages.append(package)
    return packages

# Function to update all packages
def update_packages(packages):
    for package in packages:
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', package])

# Get the list of outdated packages and update them
outdated_packages = get_outdated_packages()
update_packages(outdated_packages)


NameError: name 'sys' is not defined